In [1]:
import h2o
import os

#start the h2o cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.77-b03, mixed mode, sharing)
  Starting server from C:\Users\Renzo\AppData\Local\Programs\Python\Python38-32\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Renzo\AppData\Local\Temp\tmph56mikz9
  JVM stdout: C:\Users\Renzo\AppData\Local\Temp\tmph56mikz9\h2o_Renzo_started_from_python.out
  JVM stderr: C:\Users\Renzo\AppData\Local\Temp\tmph56mikz9\h2o_Renzo_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,America/Denver
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.2
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_Renzo_jh7i0g
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,247.5 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [2]:
#Check if file exists
if not os.path.exists("./data_input/LoanStats3a.csv"):
    input_csv = "https://s3-us-west-2.amazonaws.com/h2o-tutorials/data/topics/lending/lending_club/LoanStats3a.csv"

loans = h2o.import_file(input_csv,
                        col_types = {
                            "int_rate":"string",
                            "revol_util":"string",
                            "emp_length":"string",
                            "verification_status":"string"
                        })

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
num_unfiltered_loans = loans.dim[0]
loans["loan_status"].table().head(15)

loan_status,Count
Charged Off,5435
Current,3351
Default,7
Does not meet the credit policy. Status:Charged Off,761
Does not meet the credit policy. Status:Current,53
Does not meet the credit policy. Status:Fully Paid,1933
Does not meet the credit policy. Status:In Grace Period,2
Fully Paid,30843
In Grace Period,60
Late (16-30 days),16


In [ ]:

h2o.cluster().shutdown(prompt = True)

In [6]:
#Filtering loans, remove those in process

ongoing_status = ["Current",
                  "In Grace Period",
                  "Late (16-30 days)",
                  "Late (31-120 days)",
                  "Does not meet the credit policy.  Status:Current",
                  "Does not meet the credit policy.  Status:In Grace Period"]

loans = loans[~loans["loan_status"].isin(ongoing_status)]
loans["loan_status"].table().head(20)


loan_status,Count
Charged Off,5435
Default,7
Does not meet the credit policy. Status:Charged Off,761
Does not meet the credit policy. Status:Fully Paid,1933
Fully Paid,30843


In [14]:
#Create Response Column

fully_paid = ["Fully Paid",
              "Does not meet the credit policy. Status:Fully Paid"]
loans["bad_loan"] = ~(loans["loan_status"].isin(fully_paid))
loans["bad_loan"] = loans["bad_loan"].asfactor()


In [22]:
text_to_numeric = ["int_rate", "revol_util", "emp_length"]

#change variables - text to numeric
loans["int_rate"] = loans["int_rate"].gsub(pattern = "%", replacement = "")
loans["int_rate"] = loans["int_rate"].trim()
loans["int_rate"] = loans["int_rate"].asnumeric()

loans["revol_util"] = loans["revol_util"].gsub(pattern = "%", replacement = "")
loans["revol_util"] = loans["revol_util"].trim()
loans["revol_util"] = loans["revol_util"].asnumeric()


loans["emp_length"] = loans["emp_length"].gsub(pattern = "([ ]*+[a-zA-Z].*)|(n/a)", replacement = "")
loans["emp_length"] = loans["emp_length"].trim()
loans["emp_length"] = loans["emp_length"].gsub(pattern = "< 1", replacement = "0")
loans["emp_length"] = loans["emp_length"].gsub(pattern = "10\\+", replacement = "10")
loans["emp_length"] = loans["emp_length"].asnumeric()

loans[["int_rate", "revol_util", "emp_length"]].head()


int_rate,revol_util,emp_length
10.65,83.7,10
15.27,9.4,0
15.96,98.5,10
13.49,21,10
7.9,28.3,3
18.64,87.5,9
21.28,32.6,4
12.69,36.5,0
14.65,20.6,5
12.69,67.1,10


In [40]:
%matplotlib inline

#print("Bad Loans")
#loans[loans["bad_loan"] == 1, "int_rate"].na_omit()#.hist()

#print("Good Loans")
#loans[loans["bad_loan"] == 0, "int_rate"].hist()



Bad Loans


H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: breaks must be a positive number
  Request: POST /99/Rapids
    data: {'ast': "(tmp= py_54_sid_9819 (hist (tmp= py_53_sid_9819 (na.omit (rows (cols_py py_34_sid_9819 'int_rate') (== (cols_py py_34_sid_9819 'bad_loan') 1)))) 'sturges'))", 'session_id': '_sid_9819'}


In [45]:
loans["verification_status"] = loans["verification_status"].sub(pattern="VERIFIED - income source", 
                                                                replacement="verified")
loans["verification_status"] = loans["verification_status"].sub(pattern="VERIFIED - income", 
                                                                replacement="verified")
loans["verification_status"] = loans["verification_status"].asfactor()

loans["verification_status"].table()

verification_status,Count
not verified,17947
verified,21032
